<a href="https://colab.research.google.com/github/van-dang/DMRI-FEM-Cloud/blob/master/DiscontinuousInitialCondition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About the code

In [0]:
# This program solves the Bloch-Torrey equation applied to computational diffusion MRI using 
# the finite element method coupled with the theta-method for the spatial discretization.

# The scope of usage: 
# (1) Single domains, Multilayered structures, manifolds
# (2) Membrane permeability for internal interfaces
#     Artificial permeability at the external interfaces
# (3) pure homogeneous Neumann BCs, (4) pseudo-periodic BCs

# Copyright (C) 2019 Van-Dang Nguyen (vdnguyen@kth.se)

# This file is part of DOLFIN.

# DOLFIN is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# DOLFIN is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU Lesser General Public License for more details.

# You should have received a copy of the GNU Lesser General Public License
# along with DOLFIN. If not, see <http://www.gnu.org/licenses/>.

# First added:  2017-10-10
# Last changed: 2019-04-25

# This demo is maintained by Van-Dang Nguyen
# Please report possible problems to vdnguyen@kth.se

# Setting a working environment with FEniCS

In [4]:
try:
    from google.colab import files
except:
    print("Running on a local runtime");
    
import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
    
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *

gmsh_dir=!which gmsh
if len(gmsh_dir)==0:
  !sudo apt-get update
  !sudo apt-get install -y gmsh
      
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time

import os, sys, shutil

clear_output()

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

# Disable warnings
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("rothemain.rothe_utils")
logging.getLogger('UFL').setLevel(logging.WARNING)
logging.getLogger('FFC').setLevel(logging.WARNING)

import warnings
warnings.filterwarnings("ignore")

dolfin version: 2019.1.0


#Load pre-defined functions

In [5]:
import sympy as sp  
exists = os.path.isfile('DmriFemLib.py')
isupdate = False
if (exists==False or isupdate==True):
    if isupdate==True:
        !rm DmriFemLib.py
    print("Load pre-defined functions from GitHub")
    !wget --quiet https://raw.githubusercontent.com/van-dang/DMRI-FEM-Cloud/master/DmriFemLib.py
from DmriFemLib import *
      

Load pre-defined functions from GitHub


# Working on the mesh and submesh

In [0]:
geo_choice = 2
################################################################################
############## Create two-layered disk using mshr in FEniCS ####################
if geo_choice == 1:
    R1, R2 = 5, 10;
    origin = Point(0.,0.)
    circle = Circle(origin, R1, segments=32)
    domain = Circle(origin, R2, segments=32)
    domain.set_subdomain(1, circle)
    mymesh = generate_mesh(domain, 15) # 15 is the resolution
    cmpt_mesh = generate_mesh(circle, 15)
    evengroup=[]
    oddgroup=[cmpt_mesh]
    phase, partion_list, partition_marker = CreatePhaseFunc(mymesh, evengroup, oddgroup, None)

################################################################################
############## Create multilayered domains using gmsh ##########################
if geo_choice == 2:
    mesh_name = "multi_layered_disk"
    # mesh_name = "multi_layered_cylinder"
    # mesh_name = "multi_layered_sphere"
    # mesh_name = "multi_layered_torus"
    is_partition_function_exist = os.path.isfile('GetPartitionMarkers.py')
    if is_partition_function_exist==False:
        !wget --quiet https://raw.githubusercontent.com/van-dang/DMRI-FEM-Cloud/mesh/GetPartitionMarkers.py

    is_geo_file_exist = os.path.isfile(mesh_name+'.geo')  
    if is_geo_file_exist==False:
        !wget --quiet https://raw.githubusercontent.com/van-dang/DMRI-FEM-Cloud/mesh/$mesh_name''.geo

    # Modify .geo file from 4 layers to 3 layers      
    !sed -i 's/5, 7.5, 10, 13/5, 7.5, 10/g' $mesh_name''.geo
      
    # Create mesh from geo file by gmsh
    !gmsh -3 $mesh_name''.geo -o $mesh_name''.msh
    
    # Convert .msh to .xml using dolfin-convert
    !dolfin-convert $mesh_name''.msh $mesh_name''.xml
    # clear_output()

    mymesh = Mesh(mesh_name+".xml");  

    from GetPartitionMarkers import *

    GetPartitionMarkers(mesh_name+".msh", "pmk_"+mesh_name+".xml")

    partition_marker = MeshFunction("size_t", mymesh, mymesh.topology().dim())

    File("pmk_"+mesh_name+".xml")>>partition_marker
    
    phase, partion_list = CreatePhaseFunc(mymesh, [], [], partition_marker)    

################################################################################
############## Download the existing mesh and submesh ##########################
if geo_choice == 3:
    is_file_exist = os.path.isfile("multi_layer_torus.xml")  
    if is_file_exist==False:
        !wget --quiet https://raw.githubusercontent.com/van-dang/DMRI-FEM-Cloud/master/comri/meshes/multi_layer_torus.xml.zip
        !wget --quiet https://raw.githubusercontent.com/van-dang/DMRI-FEM-Cloud/master/comri/meshes/multi_layer_torus_compt1.xml.zip
        !unzip -q multi_layer_torus.xml.zip
        !unzip -q multi_layer_torus_compt1.xml.zip
        !rm -rf *.zip *MACOSX

    mymesh = Mesh("multi_layer_torus.xml");  
    cmpt_mesh = Mesh('multi_layer_torus_compt1.xml')
    
    phase, partion_list, partition_marker = CreatePhaseFunc(mymesh, [], [cmpt_mesh], None)

################################################################################
############## Save, Plot phase functions and submeshes to verify ##############
File("Phase.pvd")<<phase
print("Partition markers:", partion_list)
if mymesh.topology().dim()==2:
    plt.subplot(1, 2, 2); plot(partition_marker, cmap="coolwarm",  title="Partition marker");

Info    : Running 'gmsh -3 multi_layered_disk.geo -o multi_layered_disk.msh' [Gmsh 3.0.6, 1 node, max. 1 thread]
Info    : Started on Tue Jul 23 15:49:40 2019
Info    : Reading 'multi_layered_disk.geo'...
Number compartments 3
compartment 0, radius 5
compartment 1, radius 7.5
Info    : Starting subloop 1 in Line Loop 13 (are you sure about this?)
compartment 2, radius 10
Info    : Starting subloop 1 in Line Loop 16 (are you sure about this?)
Info    : Finalized high order topology of periodic connections
Info    : Meshing 1D...
Info    : Meshing curve 1 (Circle)
Info    : Meshing curve 2 (Circle)
Info    : Meshing curve 4 (Circle)
Info    : Meshing curve 5 (Circle)
Info    : Meshing curve 7 (Circle)
Info    : Meshing curve 8 (Circle)
Info    : Done meshing 1D (0.000889 s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Delaunay)
Info    : Meshing surface 2 (Plane, Delaunay)
Info    : Meshing surface 3 (Plane, Delaunay)
Info    : Done meshing 2D (0.017252 s)
Info    : 490 ve

# Solve the Bloch-Torrey equation

In [0]:
mri_simu = MRI_simulation()
mri_para = MRI_parameters()

#################################################################################
#########################  Pre-defined parameters ###############################
mri_para.stype='PGSE'                                # sequence type

bvalues = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000,10000]

for bvalue in bvalues:
    mri_para.bvalue = bvalue;                              # bvalue
    mri_para.delta, mri_para.Delta = 10600, 43100        # time sequence
    mri_para.set_gradient_dir(mymesh, 1, 0, 0)           # gradient direction
    mri_para.T = mri_para.Delta+mri_para.delta
    mri_para.fs_sym = sp.Piecewise(
                    (  1., mri_para.s < mri_para.delta ),
                    (  0., mri_para.s < mri_para.Delta ),
                    ( -1., mri_para.s < mri_para.T ),
                    (  0., True )  
                ) 

    mri_para.Apply()
    mri_simu.k = 200;                                    # time-step size
    mri_simu.nskip = 5;                                  # frequency to print ouputs

    mydomain = MyDomain(mymesh, mri_para)
    mydomain.phase = phase
    mydomain.PeriodicDir = [0, 0, 0];            # Direction of the periodicity
    mydomain.IsDomainPeriodic = False             # Confirm if the mesh if periodic
    mydomain.IsDomainMultiple = True             # Confirm if the mesh is multiple
    mydomain.kappa = 5e-5                        # Permeability
    ################################################################################

    mydomain.Apply()   # Call Apply before setting the diffusion tensor

    ################################################################################
    # Impose the diffusion coefficient
    mydomain.D0 = 3e-3
    mydomain.D = mydomain.D0;

    ################ Initialize discontinuous initial conditions
    IC_array = [1, 1, 1];
    dofmap_DG = mydomain.V_DG.dofmap()
    disc_ic = Function(mydomain.V_DG);
    for cell in cells(mymesh):
        p = cell.midpoint() # the coordinate of the cell center.
        cmk = partition_marker[cell.index()]
        cell_dof = dofmap_DG.cell_dofs(cell.index())
        disc_ic.vector()[cell_dof] = IC_array[cmk]; 

    disc_ic=project(disc_ic, mydomain.V)
    File("disc_ic.pvd")<<disc_ic
        
    #################################################################################
    #################################################################################

    # linsolver = PETScLUSolver("mumps")
    linsolver = KrylovSolver("bicgstab")
    #linsolver.parameters["absolute_tolerance"] = 1e-4
    #linsolver.parameters["relative_tolerance"] = 1e-3
    #linsolver.parameters["maximum_iterations"] = 1000

    mri_simu.solve(mydomain, mri_para, linsolver, disc_ic)
    ctext = '';
    if bvalue == 1000:
        ctext = "Disk R=[5, 7.5, 10]um, Delta=43.1ms, delta=10.6ms, D=3e-3 mm^2/s, kappa = 5e-5 m/s, PGSE, IC_array = [1, 1, 1]"
    PostProcessing(mydomain, mri_para, mri_simu, plt, ctext)

# Reference solutions

In [0]:
# The results were generated by the Matrix Formalism method propsed in
# D.S. Grebenkov, Pulsed-gradient spin-echo monitoring of restricted 
# diffusion in multilayered structures, J. Magn. Reson., 205 (2) (2010)


# # # # # # # # # # # # # # # # D.I.S.K # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # R=[5,7.5,10], PGSE delta=Delta=40000 # # # ## # # # # # # # # #
# # # # # # # # # # mydomain.PeriodicDir = [0, 0, 0];
# D=3e-3 mm^2/s, kappa=5e-5m/s
#  bvalue    # signal attenuation
#  0.000e+00 #  1.000e+00
#  1.000e+03 #  7.181e-01
#  2.000e+03 #  5.234e-01
#  3.000e+03 #  3.899e-01
#  4.000e+03 #  2.990e-01
#  5.000e+03 #  2.373e-01
#  6.000e+03 #  1.952e-01
#  7.000e+03 #  1.661e-01
#  8.000e+03 #  1.454e-01
#  9.000e+03 #  1.302e-01
#  1.000e+04 #  1.184e-01
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# D=[3 1 3]*1e-3 mm^2/s, kappa=1e-5m/s
#  0.000e+00 # 1.000e+00
#  1.000e+03 # 7.297e-01
#  2.000e+03 # 5.525e-01
#  3.000e+03 # 4.381e-01
#  4.000e+03 # 3.649e-01
#  5.000e+03 # 3.181e-01
#  6.000e+03 # 2.877e-01
#  7.000e+03 # 2.672e-01
#  8.000e+03 # 2.522e-01
#  9.000e+03 # 2.402e-01
#  1.000e+04 # 2.297e-01
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #



# # # # # # # # # # # # # # # S.P.H.E.R.E # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # R=[5,7.5,10], PGSE delta=Delta=40000 # # # # # # # #  # # # #
# # # # # # # # # # mydomain.PeriodicDir = [0, 0, 0];
# D=3e-3 mm^2/s, kappa=5e-5m/s
#  bvalue    # signal attenuation
#  0.000e+00 #  1.000e+00
#  1.000e+03 #  7.886e-01
#  2.000e+03 #  6.228e-01
#  3.000e+03 #  4.932e-01
#  4.000e+03 #  3.921e-01
#  5.000e+03 #  3.135e-01
#  6.000e+03 #  2.526e-01
#  7.000e+03 #  2.054e-01
#  8.000e+03 #  1.688e-01
#  9.000e+03 #  1.406e-01
#  1.000e+04 #  1.186e-01
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# D=[3 1 3]*1e-3 mm^2/s
#  0.000e+00 #  1.000e+00
#  1.000e+03 #  7.707e-01
#  2.000e+03 #  5.932e-01
#  3.000e+03 #  4.565e-01
#  4.000e+03 #  3.518e-01
#  5.000e+03 #  2.721e-01
#  6.000e+03 #  2.117e-01
#  7.000e+03 #  1.662e-01
#  8.000e+03 #  1.321e-01
#  9.000e+03 #  1.065e-01
#  1.000e+04 #  8.749e-02
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #


# # # # # # # # # # # # # # # C.Y.L.I.N.D.E.R # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # R=[5,7.5,10], PGSE delta=Delta=10000 # # # # # # # #  # # # #
# D=3e-3 mm^2/s, kappa=5e-5m/s
# gdir=[0, 1, 0]:
# 0.000e+00 #  1.000e+00
# 1.000e+03 #  4.777e-01
# 2.000e+03 #  2.996e-01
# 3.000e+03 #  2.235e-01
# 4.000e+03 #  1.784e-01
# 5.000e+03 #  1.455e-01
# 6.000e+03 #  1.200e-01
# 7.000e+03 #  1.003e-01
# 8.000e+03 #  8.513e-02
# 9.000e+03 #  7.374e-02
# 1.000e+04 #  6.531e-02
# gdir=[1., 0, 0]:
# mydomain.PeriodicDir = [1, 0, 0]: s=exp(-bvalue*D0)
# mydomain.PeriodicDir = [0, 0, 0]:
#  bvalue    # signal attenuation
#  0.000e+00 #  1.000e+00
#  1.000e+03 #  2.588e-01
#  2.000e+03 #  1.191e-01
#  3.000e+03 #  8.646e-02
#  4.000e+03 #  6.591e-02
#  5.000e+03 #  5.008e-02
#  6.000e+03 #  4.019e-02
#  7.000e+03 #  3.460e-02
#  8.000e+03 #  3.083e-02
#  9.000e+03 #  2.738e-02
#  1.000e+04 #  2.390e-02
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #